In [1]:
import pandas as pd
import io, gzip
from codecs import decode
import pysrt
import datetime as dt
import base64
import re
from tqdm import tqdm
from os import listdir
import pdb
import dill
from datetime import datetime
import numpy as np
import sys
from xmlrpc.client import ServerProxy
import os as os
import os.path
import json
import time


In [2]:
with open('credentials.pkd','rb') as fp:
    credentials = dill.load(fp)
USER_AGENT=credentials['user']
USERNAME=credentials['user']
PASS=credentials['passw']

In [3]:
OPENSUBTITLES_SERVER = 'http://api.opensubtitles.org/xml-rpc'
LANGUAGE = 'en'

xmlrpc = ServerProxy(OPENSUBTITLES_SERVER,allow_none=True)
data = xmlrpc.LogIn(USERNAME, PASS, LANGUAGE, USER_AGENT)
status=data.get('status')
if status != '200 OK':
    print('Could not log in. Something is wrong... ')
    print("data.get('status')")
    token = None
else:
    token = data.get('token')
    print('Logged in. Acquired token = {}'.format(token))
def server_stat():
    print('------------------Current Stats-----------------')
    for key,item in xmlrpc.ServerInfo(token)['download_limits'].items():
        print('{}: {}'.format(key,item))
    return
def get_down_quota():
    return xmlrpc.ServerInfo(token)['download_limits']['client_download_quota']

Logged in. Acquired token = ypA4zJI0CQWL4KCb5STyzV0qU04


In [4]:
server_stat()

------------------Current Stats-----------------
global_24h_download_limit: 200
client_ip: 196.52.2.96
limit_check_by: user_id
user_id: 5937493
client_24h_download_count: 372
client_download_quota: 628
client_24h_download_limit: 1000


In [12]:
with open('movie_DB_1_7_2018.pkd', 'rb') as fp:
    movie_database = dill.load(fp)

# Step 1 - Subtitle_ID's
Using out database of IMDB_ID's we seek 3 Subtitle_ID's in order of quality. Best one is first. We will then attempt download of the best, and if that doesnt work, the next one and so on... We do the search in batches of 500 whichis the maximum allowed.


In [5]:
def filter_one(sr_one):
    if not sr_one:
        return ()
    else:
        try:
            sr=[(item['SubDownloadsCnt'],item['SubRating'],item['IDSubtitleFile'],item['IDMovieImdb']) for item in sr_one 
             if item['SubFormat']=='srt' if not re.findall('.*[^a-zA-Z]cd[^a-zA-Z]',item['MovieReleaseName'].lower())]
            sr_sort=sorted(sr, key=lambda x: -int(x[0]))
            result=tuple([fill_id(sr_sort[0][3])]+[item[2] for item in sr_sort[0:3]])
            #pdb.set_trace()
            return result #returns a tuple 
        except:
            return ()

def fill_id(string):
    #opensubtitles removes the 'tt' and all leading 0's . this function replaces them.
    while len(string) < 7:
        string = '0' + string
    return 'tt' + string
    
def search_imdbid(list_imdbids):
    #SearchSubtitles is crappy because it also gives you 500 search results
    #useless to search for 500 movies at a time since each one might have ~100 results
    #for safety i will search for 10 at a time ~ 2000 requests ~ 40per 10seconds ~ 10-15 mins maybe 
    search_params=[{'sublanguageid':'eng','imdbid':item.strip('tt')} for item in list_imdbids]
    search_data=xmlrpc.SearchSubtitles(token,search_params)
    sr=[]
    try_num=0
    while sr==[]:
        if search_data['status'] == '200 OK':
            sd=search_data['data'] #a long list of search results by imdb_ids
            uniq_id=list(set([item['IDMovieImdb'] for item in sd]))
            sr=[filter_one([item for item in sd if item['IDMovieImdb'] == idnum]) for idnum in uniq_id] 
            #filter_one takes a list of search results for a single imdbid and returns a tuple (imdbid, subid1,subid2,subid3)
            #the above 3 lines will partition the search results sd into a list of lists where each inner list is for one imdbid
            if sr==[]: #sr will be an empty list if no results are found (it is possible)
                return []         
        else:
            if try_num==3: #gives up after 3 times
                print('Tried 3 times. Giving up.')
                return 
            try_num=try_num+1
            print('Error Code: {} Retrying in 10 seconds.'.format(search_data['status']))    
            time.sleep(10)
    return sr

# Take movie_database-queries opebsubs-returns sub_id list 

In [ ]:
#LA=list of imdb_ids to get

In [7]:
t0 = time.time()
sub_ids=[]
size=10
for i in tqdm(range(0,len(LA),size)):
    sub_ids=sub_ids+search_imdbid(LA[i:i+size])
print('\nElapsed {} seconds.'.format(time.time()-t0))    

NameError: name 'LA' is not defined

Clean the subs by removing all empty tuples which exist because some subs cannot be found. 
**And write to file.**

In [267]:
sub_id_clean=[item for item in sub_ids if item!=()]
with open('sub_id_temp.pkd', 'wb') as fp:
    dill.dump(sub_id_clean, fp)

In [5]:
with open('sub_id_clean.pkd', 'rb') as fp:
    sub_id_clean = dill.load(fp)
print('Number of sub_ids {}'.format(len(sub_id_clean)))

Number of sub_ids 16893


# Misc Functions

In [13]:
def get_title(imdb_id):
    try:
        ind=[item['imdb_id'] for item in movie_database].index(imdb_id)
        return movie_database[ind]['title'] 
    except:
        return 'Not Found.'

# Downloading the subs

In [14]:
def down_sub(sub_id):
    downsub=xmlrpc.DownloadSubtitles(token, [sub_id])
    if downsub['status'] != '200 OK':
        print('Error Code: {}'.format(downsub['status']))
        server_stat()
        sys.exit('Leaving this world now.')
    return downsub

def decode_sub(enc_subfile):
    dec_data=decode(bytearray(enc_subfile['data'][0]['data'].encode("utf-8")),'base64')
    try:
        sub_txt=gzip.GzipFile(fileobj=io.BytesIO(dec_data)).read().decode("utf-8")       
    except:
        sub_txt=gzip.GzipFile(fileobj=io.BytesIO(dec_data)).read().decode("ISO-8859-1")   
    return sub_txt

def write_to_file(file_name, file_content):
    #writes file_content into an srt file in the subs_storage subfolder of current path
    #if exists prints message
    #if os.path.isfile(os.getcwd()+'\\subs_storage\\'+ file_name + '.srt'):
    #    print('Sub '+ file_name + ' exists already.')
    #else:
    with open(os.getcwd()+'\\subs_storage\\'+ file_name +'.srt','wb') as fp:
        fp.write(file_content.encode('utf-8'))

# THIS PART ACTIVELY DOWNLOADS
Checks to see what *.srt files exist in the folder. 
1. If they are there nothing happens.
2. If they are not there then it uses sub_id_clean file to get the best sub listing. 

<img src="http://clipartix.com/wp-content/uploads/2017/06/Stop-sign-clip-art-microsoft-free-clipart-images-2.png" alt="Le bag of words" title="The bag of words" />

**RUNNING THE NEXT CELL WILL CAUSE DOWNLOADING TO OCCUR. YOU ONLY HAVE 1000 PER DAY.**
**MAKE SURE EVERYTHING IS GOOD BEFORE RUNINNG**

In [8]:
len(sub_id_clean)

16893

In [ ]:
for item in sub_id_clean[3000:]:
    if os.path.isfile(os.getcwd()+'\\subs_storage\\'+ item[0] + '.srt'):
        continue
        #print('SUB_ID:{:<10} IMDB_ID:{} exists ---- SKIPPING FILE Title: {}'.format(item[1],item[0],get_title(item[0])))
    else:
        print('SUB_ID:{:<10} IMDB_ID:{} downloading ------------- Title: {}'.format(item[1],item[0],get_title(item[0])))
        write_to_file(item[0],decode_sub(down_sub(item[1])))

SUB_ID:1951926039 IMDB_ID:tt0214641 downloading ------------- Title: DragonHeart: A New Beginning
SUB_ID:1955246959 IMDB_ID:tt3608646 downloading ------------- Title: Mythica: The Necromancer
SUB_ID:1953049011 IMDB_ID:tt1687901 downloading ------------- Title: The Awakening
SUB_ID:1952140185 IMDB_ID:tt0257076 downloading ------------- Title: S.W.A.T.
SUB_ID:1955123647 IMDB_ID:tt1528854 downloading ------------- Title: Daddy's Home
SUB_ID:1954663126 IMDB_ID:tt4296026 downloading ------------- Title: Avengers Grimm
SUB_ID:1952337814 IMDB_ID:tt0947810 downloading ------------- Title: Green Zone
SUB_ID:1955453768 IMDB_ID:tt2671706 downloading ------------- Title: Fences
SUB_ID:1955573382 IMDB_ID:tt4158096 downloading ------------- Title: Free Fire
SUB_ID:1952054190 IMDB_ID:tt0814314 downloading ------------- Title: Seven Pounds
SUB_ID:1954603162 IMDB_ID:tt2515030 downloading ------------- Title: Escobar: Paradise Lost
SUB_ID:297021     IMDB_ID:tt0377818 downloading ------------- Title: The

# Parsing SRT Files

In [326]:
work_dir=os.getcwd()+'\\subs_storage\\'
def read_srt(imdb_id):
    if os.path.isfile(work_dir+imdb_id + '.srt'):
        with open(work_dir+imdb_id+'.srt','rb') as fp:
            test=fp.read().decode('utf-8')
        return test
    else:
        pass

def fparse1(string):
    if string:
        #string=re.sub(r'<i>','',string)    
        #string=re.sub(r'</i>','',string)
        string=re.sub('<[^>]*.','',string) #kills anything inside < CACA >
        string=re.sub(r'\ufeff','',string)
        #string=re.sub('^[^(1)]*','',string) #removes stuff before the 1
        string=re.sub('^[^\a]*?1\r\n00:','1\r\n00:',string) #removes stuff before the 1 betterly
        return string
    else:
        pass

def srt_conv(subs):
    if subs:
        starttimes=[]
        endtimes=[]
        words=[]
        indexes=[]
        #"-->" in line
        subs=io.StringIO(subs)
        prevLineWasWords =False
        prevLineWasTime =False
        for line in subs:
            line=line.strip()
            if "-->" in line:
                starttimes.append(line[:8])
                endtimes.append(line[17:25])
                prevLineWasTime =True
            elif line.strip().isdigit() and not prevLineWasTime:
                indexes.append(int(line))
                prevLineWasTime=False
            else:
            #elif len(line) > 1:
                prevLineWasTime = False
                if prevLineWasWords:
                        words[len(words)-1] = words[len(words)-1] + " " + line
                else:
                        words.append(line)
                prevLineWasWords = True
                continue
            prevLineWasWords = False  
        
        #trim because sometimes a file will end without a last tine.
        trim_by=min([len(starttimes),len(endtimes),len(words)])
        starttimes=starttimes[0:trim_by]
        endtimes=endtimes[0:trim_by]
        words=words[0:trim_by]
        
        return starttimes, endtimes, words, indexes
    else:
        pass
    
def watch_output(clean_srt):
    subs=io.StringIO(clean_srt)
    print('=== is number, ~~~ is text, --- is time index. \n')
    prevLineWasTime=False
    for line in subs:
        if "-->" in line:
            prevLineWasTime=True
            print('----'+line)
        elif line.strip().isdigit() and not prevLineWasTime:
            prevLineWasTime=False
            print('====='+line)
        else:
        #elif len(line)>1:
            prevLineWasTime=False
            print('~~~~'+line)
    return

def s_time(string):
    return datetime.strptime(string, '%H:%M:%S')

def make_df(imdb_id):
    srt1=fparse1(read_srt(imdb_id))
    if srt1:
        a,b,c,_=srt_conv(srt1)

        imdb_idi=[imdb_id]
        linfo=['StartTime','EndTime','Line']
        index=pd.MultiIndex.from_product([imdb_idi,linfo],names=['IDs','LData'])
        data_sub=pd.DataFrame(columns=index)


        s1 = pd.Series([s_time(item) for item in a])
        s2 = pd.Series([s_time(item) for item in b])
        s3 = pd.Series(c)
        data_sub.loc[:,(imdb_id,'StartTime')] =s1.values
        data_sub.loc[:,(imdb_id,'EndTime')] = s2.values
        data_sub.loc[:,(imdb_id,'Line')] = s3.values

        return data_sub
    else:
        pass
    
def append_pd(df1,df):
    tes=pd.concat([df1,df], axis=1)
    tes=tes.fillna('')
    return tes

In [328]:
aid=[item.split('.')[0] for item in listdir(work_dir) if 'tt' in item]

In [329]:
len(aid)

1999

In [333]:
t=time.time()
full_data=pd.DataFrame()
for i in range(70,len(aid)):
    print('No. {} - adding {}'.format(i,aid[i]))
    full_data=append_pd(full_data,make_df(aid[i]))
t1=time.time()
print('elapsed time {} seconds'.format(t1-t))

No. 70 - adding tt0031359
No. 71 - adding tt0031377
No. 72 - adding tt0031507
No. 73 - adding tt0031851
No. 74 - adding tt0032022
No. 75 - adding tt0032143
No. 76 - adding tt0032194
No. 77 - adding tt0032234
No. 78 - adding tt0032285
No. 79 - adding tt0032390
No. 80 - adding tt0032520
No. 81 - adding tt0032554
No. 82 - adding tt0032728
No. 83 - adding tt0032811
No. 84 - adding tt0033226
No. 85 - adding tt0033253
No. 86 - adding tt0033388
No. 87 - adding tt0033627
No. 88 - adding tt0033740
No. 89 - adding tt0033874
No. 90 - adding tt0033951
No. 91 - adding tt0034172
No. 92 - adding tt0034175
No. 93 - adding tt0034236
No. 94 - adding tt0034297
No. 95 - adding tt0034384
No. 96 - adding tt0034399
No. 97 - adding tt0034522
No. 98 - adding tt0034746
No. 99 - adding tt0034786
No. 100 - adding tt0034798
No. 101 - adding tt0034922
No. 102 - adding tt0035019
No. 103 - adding tt0035317
No. 104 - adding tt0035605
No. 105 - adding tt0035665
No. 106 - adding tt0035884
No. 107 - adding tt0035937
No. 

ValueError: time data ' 00:34:2' does not match format '%H:%M:%S'

In [344]:
make_df('tt0036984')

ValueError: time data ' 00:34:2' does not match format '%H:%M:%S'

In [334]:
print(re.sub('^[^\a]*?1\r\n00:','1\r\n00:',read_srt('tt0036984')))

1
00:00:17,648 --> 00:00:21,277
Kismet

2
00:01:26,340 --> 00:01:30,935
Once upon a time when old Baghdad
was new and shiny,

3
00:01:31,079 --> 00:01:34,114
there was a beggar and what a rascal!

4
00:01:34,593 --> 00:01:36,996
He called himself the king of beggars.

5
00:01:37,168 --> 00:01:40,260
Having convinced himself that he was
a king of some kind,

6
00:01:40,547 --> 00:01:43,582
He'd roam the streets at night
seeking adventure

7
00:01:43,697 --> 00:01:45,612
disguised as a prince.

8
00:01:45,660 --> 00:01:49,394
In a great palace in the city
lived a great lady.

9
00:01:49,461 --> 00:01:50,581
A beautiful lady.

10
00:01:50,705 --> 00:01:53,558
And she had quite a time
with this prince.

11
00:01:53,692 --> 00:01:57,091
Of course, she didn't know
that he was only a beggar.

12
00:01:57,196 --> 00:02:00,269
Her lord and master was
the Grand Vizier.

13
00:02:00,317 --> 00:02:03,161
A very powerful man.

14
00:02:03,209 --> 00:02:06,521
When you know him better,
you won't won

This way.


In [246]:
ttu=io.StringIO(tt)
[line for line in ttu if line.strip()=='1']

['1\n']

In [356]:
print(read_srt('tt0036984'))

1
00:00:17,648 --> 00:00:21,277
Kismet

2
00:01:26,340 --> 00:01:30,935
Once upon a time when old Baghdad
was new and shiny,

3
00:01:31,079 --> 00:01:34,114
there was a beggar and what a rascal!

4
00:01:34,593 --> 00:01:36,996
He called himself the king of beggars.

5
00:01:37,168 --> 00:01:40,260
Having convinced himself that he was
a king of some kind,

6
00:01:40,547 --> 00:01:43,582
He'd roam the streets at night
seeking adventure

7
00:01:43,697 --> 00:01:45,612
disguised as a prince.

8
00:01:45,660 --> 00:01:49,394
In a great palace in the city
lived a great lady.

9
00:01:49,461 --> 00:01:50,581
A beautiful lady.

10
00:01:50,705 --> 00:01:53,558
And she had quite a time
with this prince.

11
00:01:53,692 --> 00:01:57,091
Of course, she didn't know
that he was only a beggar.

12
00:01:57,196 --> 00:02:00,269
Her lord and master was
the Grand Vizier.

13
00:02:00,317 --> 00:02:03,161
A very powerful man.

14
00:02:03,209 --> 00:02:06,521
When you know him better,
you won't won

This way.


In [355]:
print(test[39820:39900])

 good.

539
00:34:18,315 --> 00:34:20,972
Water, water, it's good.
I never 


In [352]:
test

'1\r\n00:00:17,648 --> 00:00:21,277\r\nKismet\r\n\r\n2\r\n00:01:26,340 --> 00:01:30,935\r\nOnce upon a time when old Baghdad\r\nwas new and shiny,\r\n\r\n3\r\n00:01:31,079 --> 00:01:34,114\r\nthere was a beggar and what a rascal!\r\n\r\n4\r\n00:01:34,593 --> 00:01:36,996\r\nHe called himself the king of beggars.\r\n\r\n5\r\n00:01:37,168 --> 00:01:40,260\r\nHaving convinced himself that he was\r\na king of some kind,\r\n\r\n6\r\n00:01:40,547 --> 00:01:43,582\r\nHe\'d roam the streets at night\r\nseeking adventure\r\n\r\n7\r\n00:01:43,697 --> 00:01:45,612\r\ndisguised as a prince.\r\n\r\n8\r\n00:01:45,660 --> 00:01:49,394\r\nIn a great palace in the city\r\nlived a great lady.\r\n\r\n9\r\n00:01:49,461 --> 00:01:50,581\r\nA beautiful lady.\r\n\r\n10\r\n00:01:50,705 --> 00:01:53,558\r\nAnd she had quite a time\r\nwith this prince.\r\n\r\n11\r\n00:01:53,692 --> 00:01:57,091\r\nOf course, she didn\'t know\r\nthat he was only a beggar.\r\n\r\n12\r\n00:01:57,196 --> 00:02:00,269\r\nHer lord an

In [345]:
test=(fparse1(read_srt('tt0036984')))

In [346]:
print(test)

1
00:00:17,648 --> 00:00:21,277
Kismet

2
00:01:26,340 --> 00:01:30,935
Once upon a time when old Baghdad
was new and shiny,

3
00:01:31,079 --> 00:01:34,114
there was a beggar and what a rascal!

4
00:01:34,593 --> 00:01:36,996
He called himself the king of beggars.

5
00:01:37,168 --> 00:01:40,260
Having convinced himself that he was
a king of some kind,

6
00:01:40,547 --> 00:01:43,582
He'd roam the streets at night
seeking adventure

7
00:01:43,697 --> 00:01:45,612
disguised as a prince.

8
00:01:45,660 --> 00:01:49,394
In a great palace in the city
lived a great lady.

9
00:01:49,461 --> 00:01:50,581
A beautiful lady.

10
00:01:50,705 --> 00:01:53,558
And she had quite a time
with this prince.

11
00:01:53,692 --> 00:01:57,091
Of course, she didn't know
that he was only a beggar.

12
00:01:57,196 --> 00:02:00,269
Her lord and master was
the Grand Vizier.

13
00:02:00,317 --> 00:02:03,161
A very powerful man.

14
00:02:03,209 --> 00:02:06,521
When you know him better,
you won't won

This way.


In [347]:
a,b,c,_=srt_conv(test)

In [348]:
for item in b:
    s_time(item)

ValueError: time data ' 00:34:2' does not match format '%H:%M:%S'

In [205]:
[item for item in zip(a[0:1000],b[0:1000],c[0:1000])]

[('00:01:27', '00:01:29', 'OK, Clark. '),
 ('00:01:43', '00:01:44', 'Sit down, Clark. '),
 ('00:01:47', '00:01:49', 'I said, sit down. '),
 ('00:01:57', '00:02:01', "Here's the money you have coming: $222.35 "),
 ('00:02:02', '00:02:03', 'Sign here. '),
 ('00:02:04',
  '00:02:08',
  'You also get a railroad ticket anywhere in the United States. '),
 ('00:02:09', '00:02:11', "The state's getting off cheap. "),
 ('00:02:12', '00:02:14', 'All I need is a nickel for a streetcar. '),
 ('00:02:14', '00:02:17', "I didn't know you had any family left in town. "),
 ('00:02:17', '00:02:18', "I haven't. "),
 ('00:02:22', '00:02:24', "It's not easy to get along alone, Bill. "),
 ('00:02:24', '00:02:25', 'So, uh... '),
 ('00:02:26', '00:02:27', "Here's the fare. "),
 ('00:02:27', '00:02:29', "It's a dime now. "),
 ('00:02:29', '00:02:31', 'A lot of things have changed out there. '),
 ('00:02:32', '00:02:35', 'A lot of us have changed in here too. '),
 ('00:02:35', '00:02:37', 'You figured to change

In [341]:
watch_output(test[0:500])

=== is number, ~~~ is text, --- is time index. 

=====1

----00:00:17,648 --> 00:00:21,277

~~~~Kismet

~~~~

=====2

----00:01:26,340 --> 00:01:30,935

~~~~Once upon a time when old Baghdad

~~~~was new and shiny,

~~~~

=====3

----00:01:31,079 --> 00:01:34,114

~~~~there was a beggar and what a rascal!

~~~~

=====4

----00:01:34,593 --> 00:01:36,996

~~~~He called himself the king of beggars.

~~~~

=====5

----00:01:37,168 --> 00:01:40,260

~~~~Having convinced himself that he was

~~~~a king of some kind,

~~~~

=====6

----00:01:40,547 --> 00:01:43,582

~~~~He'd roam the streets at night

~~~~seeking adventure

~~~~

=====7

----00:01:43,697 --> 00:01:45,61


In [70]:
def download_sub(best_sub_id):
    if best_sub_id:
        #best_sub_id='1952195191'
        down_data=xmlrpc.DownloadSubtitles(token, [best_sub_id])
        if down_data['status'] !='200 OK':
            print('--------------------- Error Message:')
            print(down_data['status'])
            exit("Exiting Program Now. Come back Tomorrow. ")
        return down_data
    else:
        return False
    
def decode_parse(down_data,imdb_id):
    if down_data:
        compressed_subdata = decode(bytearray(down_data['data'][0]['data'].encode("utf-8")),'base64')
        sub_text = gzip.GzipFile(fileobj=io.BytesIO(compressed_subdata)).read()
        #parse it and store it into a DataFrame with three columns, starttime, endtime, and line
        sub_text_str = str(sub_text)
        snew = sub_text_str.replace("\\n","\n").replace("\\r","\r").replace("</i>","").replace("<i>-","").replace("<i>","").replace("\\","")
        subs=list(filter(None, snew.split('\r\n')))
        subs=[x for x in subs if not x.isdigit()]
        if subs[0][0]=='b':
            subs.pop(0)

        starttimes=[]
        endtimes=[]
        words=[]
        #"-->" in line
        prevLineWasWords =False
        for line in subs:
            line=line.strip()
            if "-->" in line:
                starttimes.append(line[:8])
                endtimes.append(line[17:25])
            elif len(line) > 1:
                    if prevLineWasWords:
                            words[len(words)-1] = words[len(words)-1] + " " + line
                    else:
                            words.append(line)
                    prevLineWasWords = True
                    continue
            prevLineWasWords = False  

        #this needs to be made better but for now we need these to be same length so it will have to do
        trim_by=min([len(starttimes),len(endtimes),len(words)])
        starttimes=starttimes[0:trim_by]
        endtimes=endtimes[0:trim_by]
        words=words[0:trim_by]

        imdb_idi=[imdb_id]
        linfo=['StartTime','EndTime','Line']
        index=pd.MultiIndex.from_product([imdb_idi,linfo],names=['IDs','LData'])
        plines=pd.DataFrame(columns=index)


        s1 = pd.Series(starttimes)
        s2 = pd.Series(endtimes)
        s3 = pd.Series(words)
        plines.loc[:,(imdb_id,'StartTime')] = s1.values
        plines.loc[:,(imdb_id,'EndTime')] = s2.values
        plines.loc[:,(imdb_id,'Line')] = s3.values
        return plines
    else:
        return False

#def populate_db(list):
    #input is a list of IMDB_IDS
    #open existing database of subtitles
    #check if each id is in there
        #if not download it and add it to the list (do this 20 at a time)
        #if yes then move on

In [16]:
##open existing database and download shit from it
if os.path.isfile('movbaseB.h5'):
    movdb=pd.read_hdf('movbaseB.h5', 'imdb_id')
else:
    print('Cannot find file movbaseB!')

In [10]:
batch=movdb[1000:1350]
len(batch)

350

In [11]:
#run this PER GENRE
#input is a list of imdb_id's which can come from whatever
#probably should come from a subset of movdb
if os.path.isfile('Sub_data.h5'):
    Sub_data=pd.read_hdf('Sub_data.h5', 'subs')
else:
    print('Cannot find file Sub_data! Making new one.')

for item in batch['imdb_id']:
    if item not in Sub_data.columns.levels[0]:
            one_sub=decode_parse(download_sub(find_best_file(search_imdbid(item),1)),item)
            try:
                if not one_sub:
                    print('COULDNT FIND SUBS FOR {}.   Title:  {} '.format(item,batch[batch['imdb_id'] == item]['Title'].values[0]))
                    continue
            except:
                if len(one_sub) < 2:
                    one_sub=decode_parse(download_sub(find_best_file(search_imdbid(item),2)),item)

                    if len(one_sub) < 2:
                        one_sub=decode_parse(download_sub(find_best_file(search_imdbid(item),3)),item)

                        if len(one_sub) <2:
                            print('First 3 subs for {} were bad. Not going any further. Title:  {} '.format(item,batch[batch['imdb_id'] == item]['Title'].values[0]))
                            continue



                Sub_data=pd.concat([Sub_data,one_sub], axis=1)
                Sub_data=Sub_data.fillna('')
                Sub_data.to_hdf('Sub_data.h5', 'subs', mode='w')
                print('Downloaded subs for {}.   Title:  {} '.format(item,batch[batch['imdb_id'] == item]['Title'].values[0]))

    else:
        print('Encountered double of {}. Title:  {}'.format(item,batch[batch['imdb_id'] == item]['Title'].values[0]))
        continue
        
print('Done. Writing Sub_data to file.')

Downloaded subs for 3387542.   Title:  The Forest 
Downloaded subs for 0130018.   Title:  I Still Know What You Did Last Summer 
Downloaded subs for 1311067.   Title:  Halloween II 
Downloaded subs for 0433386.   Title:  The Grudge 2 
Downloaded subs for 0099528.   Title:  The Exorcist III 
Downloaded subs for 0097368.   Title:  The Fly II 
Downloaded subs for 0450345.   Title:  The Wicker Man 
Downloaded subs for 0091419.   Title:  Little Shop of Horrors 
Downloaded subs for 1053859.   Title:  The Grudge 3 
Downloaded subs for 0192731.   Title:  Urban Legends: Final Cut 
Downloaded subs for 0220506.   Title:  Halloween: Resurrection 
Downloaded subs for 0475944.   Title:  The Covenant 
Downloaded subs for 0105217.   Title:  Raising Cain 
Downloaded subs for 0155975.   Title:  Psycho 
Downloaded subs for 1270761.   Title:  Don't Be Afraid of the Dark 
Downloaded subs for 0083972.   Title:  Friday the 13th Part III 
Downloaded subs for 0096734.   Title:  The 'Burbs 
Downloaded subs for 

COULDNT FIND SUBS FOR .   Title:  The Story Of Short Stack 
Downloaded subs for 0098213.   Title:  Roger & Me 
Downloaded subs for 1664894.   Title:  Cave of Forgotten Dreams 
Downloaded subs for 0226648.   Title:  Wolves 
Downloaded subs for 0088178.   Title:  Stop Making Sense 
Downloaded subs for 1587707.   Title:  Exit Through the Gift Shop 
Downloaded subs for 0379225.   Title:  The Corporation 
Downloaded subs for 1529567.   Title:  Sea Rex 3D: Journey to a Prehistoric World 
COULDNT FIND SUBS FOR 4266692.   Title:  Drain the Bermuda Triangle 
Downloaded subs for 0100332.   Title:  Paris is Burning 
Downloaded subs for 0388789.   Title:  Born into Brothels 
COULDNT FIND SUBS FOR 3635920.   Title:  D-Day: Normandy 1944 
Downloaded subs for 0109508.   Title:  Crumb 
Downloaded subs for 0389326.   Title:  Riding Giants 
Downloaded subs for 3882074.   Title:  The Eagle Huntress 
Downloaded subs for 0342172.   Title:  Capturing the Friedmans 
Downloaded subs for 1584016.   Title:  Cat

Downloaded subs for 1322313.   Title:  Sunshine Superman 
Downloaded subs for 1112115.   Title:  King Corn 
COULDNT FIND SUBS FOR 0216731.   Title:  Edgeplay: A Film About The Runaways 
Downloaded subs for 1990352.   Title:  The Island President 
Downloaded subs for 0970521.   Title:  Turtle: The Incredible Journey 
COULDNT FIND SUBS FOR 1541666.   Title:  Circo 
COULDNT FIND SUBS FOR 1746153.   Title:  Crime After Crime 
COULDNT FIND SUBS FOR 0430916.   Title:  Beautiful Losers 
COULDNT FIND SUBS FOR .   Title:  The Story Of Short Stack 
COULDNT FIND SUBS FOR 2007995.   Title:  Screaming in High Heels: The Rise & Fall of th.. 
Downloaded subs for 3268458.   Title:  The Internet's Own Boy: The Story of Aaron Swartz 
COULDNT FIND SUBS FOR 0264802.   Title:  Live Nude Girls Unite! 
Downloaded subs for 3270538.   Title:  Requiem for the American Dream 
COULDNT FIND SUBS FOR 1276943.   Title:  Bringing Godzilla Down to Size 
COULDNT FIND SUBS FOR 2234315.   Title:  Occupy Unmasked 
COULDNT

In [12]:
read_data=pd.read_hdf('Sub_data.h5', 'subs')
len(read_data.columns.levels[0])

1601